In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, LayerNormalization
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.utils.class_weight import compute_class_weight

os.environ["TF_ENABLE_ONEDNN_OPTS"] = "1"
os.environ["TF_NUM_INTEROP_THREADS"] = "4"
os.environ["TF_NUM_INTRAOP_THREADS"] = "4"

print("✅ TensorFlow version:", tf.__version__)
print("✅ GPU available:", tf.config.list_physical_devices('GPU'))

if os.path.exists("/kaggle/input/fer2013/train"):
    BASE_PATH = "/kaggle/input/fer2013"
else:
    BASE_PATH = "fer2013" 

train_dir = os.path.join(BASE_PATH, "train")
val_dir   = os.path.join(BASE_PATH, "test")

try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("✅ Using mixed precision (AMP)")
except Exception as e:
    print("⚠️ Mixed precision not supported:", e)

img_size = 224
batch_size = 64  

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False
)

num_classes = train_gen.num_classes
print(f"✅ Found {num_classes} classes")

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights = dict(enumerate(class_weights))
print("✅ Computed class weights:", class_weights)

base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(img_size, img_size, 3)
)
base_model.trainable = False 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = LayerNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=outputs)

optimizer = AdamW(learning_rate=1e-3, weight_decay=1e-4)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

os.makedirs("models", exist_ok=True)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint("models/effnetb0_stage1.h5", monitor="val_accuracy", save_best_only=True),
]

print("\n🧠 Stage 1: Training top layers...")
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)

for layer in base_model.layers[-120:]:
    layer.trainable = True

optimizer2 = AdamW(learning_rate=1e-4, weight_decay=1e-5)
model.compile(
    optimizer=optimizer2,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

callbacks2 = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint("models/effnetb0_stage2.h5", monitor="val_accuracy", save_best_only=True),
]

print("\n🔥 Stage 2: Fine-tuning deeper layers...")
history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weights,
    callbacks=callbacks2
)

for layer in base_model.layers:
    layer.trainable = True

optimizer3 = AdamW(learning_rate=5e-5, weight_decay=1e-6)
model.compile(
    optimizer=optimizer3,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

callbacks3 = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("models/effnetb0_final.h5", monitor="val_accuracy", save_best_only=True),
]

print("\n🚀 Stage 3: Full fine-tuning...")
history3 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks3
)

model.save("models/fer2013_effnetb0_224_final.h5")
print("✅ Saved model: models/fer2013_effnetb0_224_final.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open("models/fer2013_effnetb0_224_final.tflite", "wb") as f:
    f.write(tflite_model)
print("✅ Exported quantized TFLite model.")


2025-11-05 12:26:32.516275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762345592.958013      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762345593.075042      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ TensorFlow version: 2.18.0
✅ GPU available: []
✅ Using mixed precision (AMP)


2025-11-05 12:26:59.097599: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Found 28709 images belonging to 7 classes.


Found 7178 images belonging to 7 classes.


...


113/449 ━━━━━━━━━━━━━━━━━━━━ 1:04:34 12s/step - accuracy: 0.3098 - loss: 1.7619


114/449 ━━━━━━━━━━━━━━━━━━━━ 1:04:22 12s/step - accuracy: 0.3098 - loss: 1.7621


115/449 ━━━━━━━━━━━━━━━━━━━━ 1:04:09 12s/step - accuracy: 0.3097 - loss: 1.7623


116/449 ━━━━━━━━━━━━━━━━━━━━ 1:03:57 12s/step - accuracy: 0.3097 - loss: 1.7625


117/449 ━━━━━━━━━━━━━━━━━━━━ 1:03:45 12s/step - accuracy: 0.3096 - loss: 1.7626

In [ ]:

import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

class Cast(tf.keras.layers.Layer):
    def __init__(self, dtype=tf.float32, **kwargs):
        super().__init__(**kwargs)
        self.target_dtype = dtype

    def call(self, inputs):
        return tf.cast(inputs, self.target_dtype)

    def get_config(self):
        config = super().get_config()
        config.update({"dtype": self.target_dtype})
        return config

print("\n🔁 Loading FER-trained EfficientNet model for RAF-DB fine-tuning...")

fer_model_path = "/kaggle/input/efficientnet/tensorflow2/default/1/fer2013_effnetb0_224_final.h5"
if not os.path.exists(fer_model_path):
    raise FileNotFoundError(f"❌ FER model not found at: {fer_model_path}")

model = tf.keras.models.load_model(
    fer_model_path,
    custom_objects={"Cast": Cast},
    compile=False
)
print("✅ Loaded FER model")

if os.path.exists("/kaggle/input/raf-db-dataset/DATASET/"):
    RAF_BASE = "/kaggle/input/raf-db-dataset/DATASET/"
else:
    RAF_BASE = "rafdb"

raf_train = os.path.join(RAF_BASE, "train")
raf_val = os.path.join(RAF_BASE, "test")


img_size = 224
batch_size = 32

raf_train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.25,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    fill_mode="nearest"
)

raf_val_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)

raf_train_gen = raf_train_datagen.flow_from_directory(
    raf_train,
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True
)

raf_val_gen = raf_val_datagen.flow_from_directory(
    raf_val,
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False
)

num_classes_raf = raf_train_gen.num_classes
print(f"✅ RAF-DB has {num_classes_raf} emotion classes")

raf_to_fer_map = {
    0: 6,  
    1: 2,
    2: 1,  
    3: 3,  
    4: 5, 
    5: 0,  
    6: 4 
}

def remap_generator_classes(generator):
    shifted_classes = generator.classes - generator.classes.min()
    new_classes = np.array([raf_to_fer_map[c] for c in shifted_classes])
    generator.classes = new_classes
    generator.class_indices = {cls: raf_to_fer_map[idx] for cls, idx in generator.class_indices.items()}
    return generator

raf_train_gen = remap_generator_classes(raf_train_gen)
raf_val_gen = remap_generator_classes(raf_val_gen)
print("✅ Remapped RAF-DB labels to FER2013 order (1–7 → 0–6)")

class_weights_raf = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(raf_train_gen.classes),
    y=raf_train_gen.classes
)
class_weights_raf = dict(enumerate(class_weights_raf))
print("✅ Computed RAF-DB class weights:", class_weights_raf)


if model.output_shape[-1] != num_classes_raf:
    x = model.layers[-3].output
    x = Dropout(0.25)(x)  # moderate dropout → more confident predictions
    outputs = Dense(num_classes_raf, activation='softmax', dtype='float32')(x)
    model = Model(inputs=model.input, outputs=outputs)
    print("🔁 Replaced final classification head for RAF-DB")

for layer in model.layers[-160:]:
    layer.trainable = True
print("✅ Unfrozen top 160 layers for fine-tuning")


try:
    from tensorflow.keras.optimizers.legacy import AdamW
    print("✅ Using legacy.AdamW optimizer")
except ImportError:
    from tensorflow.keras.optimizers import Adam
    AdamW = Adam
    print("⚠️ legacy.AdamW not available, using Adam")

optimizer = AdamW(learning_rate=5e-5, weight_decay=1e-6)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.0),  
    metrics=['accuracy']
)

os.makedirs("models", exist_ok=True)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint(
        filepath="models/rafdb_finetuned.weights.h5",  
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True
    ),
]


print("\n🚀 Fine-tuning FER-trained model on RAF-DB...")
history = model.fit(
    raf_train_gen,
    validation_data=raf_val_gen,
    epochs=15,
    class_weight=class_weights_raf,
    callbacks=callbacks,
    shuffle=True,
    verbose=1
)


final_path = "models/effnetb0_fer_to_rafdb_final.h5"
model.save(final_path)
print(f"✅ Saved final fine-tuned model: {final_path}")


In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf

class Cast(tf.keras.layers.Layer):
    def __init__(self, dtype=tf.float32, **kwargs):
        super().__init__(**kwargs)
        self.target_dtype = dtype
    def call(self, inputs):
        return tf.cast(inputs, self.target_dtype)
    def get_config(self):
        cfg = super().get_config()
        cfg.update({"dtype": self.target_dtype})
        return cfg

print("\n🔧 Rebuilding architecture that matches your saved weights...")

base_model = EfficientNetB0(
    include_top=False,
    weights=None,
    input_shape=(224, 224, 3),
    pooling="avg"
)

x = base_model.output
x = Dense(512, activation="relu", name="dense")(x)   # ✅ 512-unit layer
x = Dropout(0.25)(x)
outputs = Dense(7, activation="softmax", dtype="float32", name="dense_1")(x)
model = Model(inputs=base_model.input, outputs=outputs)

print("✅ Architecture matches weights file (Dense(512) → Dense(7)).")

weights_path = "models/rafdb_finetuned.weights.h5"
if not tf.io.gfile.exists(weights_path):
    raise FileNotFoundError(f"❌ Weights file not found: {weights_path}")

model.load_weights(weights_path)
print("✅ Weights loaded successfully — no mismatches detected!")

save_path = "models/effnetb0_fer_to_rafdb_final.keras"
model.save(save_path)
print(f"✅ Model rebuilt and saved at: {save_path}")

model.summary()
